## Table Export
The purpose of this script is to loop through the filtered NHD collection of lakes, generate a median reflectance table of blue, green, red, near IR, short-wave IR 1, short-wave IR 2, and others in a time-series format. These will be used later in R for processing and water clarity estimation to get a sense of it's change over time. 
Joshua L. Mincer 7/29/2020


Load in all of the needed packages and scripts:

In [1]:
import ee
import folium
import time
import logging
from datetime import datetime
import simplejson as json
import GetImageCollection as ic
import MergeChunk as mc
import Subroutines as sub
import CloudCodes as cc

ee.Initialize()

nhd = ee.FeatureCollection("users/mincej20/NHD_Filtered_addID").sort("AreSqKm", False)
gsw = ee.Image('JRC/GSW1_0/GlobalSurfaceWater').select("occurrence")

In [2]:
print(datetime.now())

2020-08-16 18:34:37.459684


For a test, let's subset the list. 

In [3]:
nhd_id = nhd.select(propertySelectors = ["system:index"], retainGeometry=False)
nhd_list = (nhd_id.toList(1_000_000)
            .map(lambda feat: ee.Feature(feat).id())
            .getInfo())
nhd_size = len(nhd_list)

In [4]:
print(nhd_list[0])
print(nhd_size)

0000000000000000001f
104713


Lets put together a group that will export each derived thing and see how long it takes. 
We have about 100,000 lakes, 1,000 times less than our subset. We'll keep this in mind. 

In [5]:
i = 0
log = open('Log_Export.txt', 'a')
dt = datetime.now()
sesh = str("Starting session: " + str(dt) + "\n")
end_sesh = "Ending session..."
log.write(sesh)

while i < 10:
    nhd_id = nhd_list[i]
    feat = nhd.filterMetadata("system:index", "equals", nhd_id)
    
    if ee.Number(feat.size()).getInfo() > 1:
        msg = "Not unique ids!"
        log.write(msg + "\n" + end_sesh + str(datetime.now()))
        print(msg)
        log.close()
        break
    else: 
        feat = ee.Feature(feat.first())
        
    geom = feat.geometry()
    name = str(ee.String(feat.get("GNIS_Nm")).getInfo())
    nhdplid = str(ee.String(feat.get("NHDPlID")).getInfo())
    msg = str(i) + "   " + nhdplid + "   " + name + "\n"
    
    print(msg)
    log.write(msg)
    try: 
        imgs = ic.getImageCollection(feat, False)
        imgs = (imgs.sort("system:time_start"))
    except: 
        err = logging.exception("message")
        msg = "Memory Error: Getting Images"
        log.write(msg + "\n", + err + "\n" + end_sesh + str(datetime.now()))
        print(msg)
        log.close()
        break

    try: 
        imgs = imgs.map(sub.noClouds).map(sub.waterIsolationSWIR(gsw, geom))
    except: 
        err = logging.exception("message")
        msg = "Memory Error: Water Isolation"
        log.write(msg + "\n", + err + "\n" + end_sesh + str(datetime.now()))
        print(msg)
        log.close()
        break
        
    try: 
        imgs = ee.ImageCollection(mc.mergeByChunk(imgs))
        imgs = imgs.map(sub.distinctMerged)

    except: 
        err = logging.exception("message")
        msg = "Memory Error: Merging"
        log.write(msg + "\n", + err + "\n" + end_sesh + str(datetime.now()))
        print(msg)
        log.close()
        break
    
    size_tab = ee.Number(imgs.size()).getInfo()
    size_dist = ee.Number(imgs.aggregate_sum("Count_Distinct")).getInfo()
    
    if size_tab != size_dist:
        msg = "Images from different satellites merged!"
        log.write(msg + "\n" + end_sesh + str(datetime.now()))
        print(msg)
        log.close()
        break

    try: 
        imgs = imgs.map(sub.countPixels(geom))
        imgs = imgs.filterMetadata("Count", "greater_than", 11) 
    except: 
        err = logging.exception("message")
        msg = "Memory Error: Counting Pixels and Filtering"
        log.write(msg + "\n", + err + "\n" + end_sesh + str(datetime.now()))
        print(msg)
        log.close()
        break

    

    try: 
        imgs = imgs.map(sub.satInt)
        table = ee.FeatureCollection(imgs.map(sub.reduceColors(feat)))

    except: 
        err = logging.exception("message")
        msg = "Memory Error: Reducing Colors"
        log.write(msg + "\n", + err + "\n" + end_sesh + str(datetime.now()))
        print(msg)
        log.close()
        break

    
    task = ee.batch.Export.table.toDrive(table, 
                                  (ee.String("Export").cat(" ").cat(ee.String(name))).getInfo(),
                                  **{'fileFormat': 'CSV',
                                     'driveFolder': 'LakeExports', 
                                     'fileNamePrefix': nhdplid})
    try:
        task.start()
        i += 1
    except: 
        msg = "Queue Full: Waiting 5 minutes..."
        log.write(msg + "\n")
        print(msg)
        err = logging.exception("message")

        print(err)
        time.sleep(300)
        
        try:
            task.start()
            i += 1
        except: 
            msg = "Failed again. Looping through again..."
            log.write(msg + "\n")
            print(msg)
            
log.close()
    

0   22000800267833   Lake Michigan



EEException: Computation timed out.